# Numerical Optimization: Traffic Optimization with Trace

## Introduction

This tutorial will guide you through the process of using `trace` to optimize parameters in a traffic simulation. The goal is to find the optimal green light durations for an intersection to minimize overall traffic delay.

## Setup and Installation

First, ensure you have the required packages installed in addition to `trace`. You can install them using pip.

    !pip install numpy uxsim

## Import Necessary Libraries

Let's start by importing the necessary libraries.

In [ ]:
import numpy as np
import uxsim as ux
import itertools
import opto.trace as trace
from opto.trace.optimizers import FunctionOptimizerV2Memory
from opto.trace.bundle import ExceptionNode

## Define Constants and Functions

We define the constants and helper functions needed for the simulation.

In [ ]:
# Define minimum and maximum values for green light durations (parameter space)
MIN_GREEN_TIME = 15
MAX_GREEN_TIME = 90

# Define the simulation parameters (in seconds)
MAX_DURATION = 1800
SIMULATION_STEP = 6

# Define the demands
def create_demand(seed, demand=0.25):
    np.random.seed(seed)
    demandDict = {}
    for n1, n2 in itertools.permutations(["W1", "E1", "N1", "S1"], 2):
        for t in range(0, MAX_DURATION, SIMULATION_STEP):
            demandDict[(n1, n2, t)] = np.random.uniform(0, demand)
    # Add extra demand for E-W direction
    for t in range(0, MAX_DURATION // 3, SIMULATION_STEP):
        demandDict[("W1", "E1", t)] += demand
    for t in range(2 * MAX_DURATION // 3, MAX_DURATION, SIMULATION_STEP):
        demandDict[("E1", "W1", t)] += demand
    return demandDict


## Using bundle to wrap the Create World Function

The create_world function sets up the traffic intersection with given green light durations.
Let us wrap the function with `trace.bundle` to let the optimizer view the entire function as a node in the traced graph.

In [ ]:
@trace.bundle(trainable=False, allow_external_dependencies=True)
def create_world(EW_time, NS_time):
    global seed
    global demand_dict

    assert EW_time >= MIN_GREEN_TIME and EW_time <= MAX_GREEN_TIME, "EW_time out of bounds."
    assert NS_time >= MIN_GREEN_TIME and NS_time <= MAX_GREEN_TIME, "NS_time out of bounds."

    W = ux.World(
        name="Grid World",
        deltan=1,
        reaction_time=1,
        tmax=MAX_DURATION,
        print_mode=0,
        save_mode=0,
        show_mode=0,
        random_seed=seed,
        duo_update_time=120,
        show_progress=0,
        vehicle_logging_timestep_interval=-1,
    )

    W1 = W.addNode("W1", -1, 0)
    E1 = W.addNode("E1", 1, 0)
    N1 = W.addNode("N1", 0, 1)
    S1 = W.addNode("S1", 0, -1)

    for k, v in demand_dict.items():
        n1, n2, t = k
        node1 = eval(n1)
        node2 = eval(n2)
        W.adddemand(node1, node2, t, t + SIMULATION_STEP, v)

    I1 = W.addNode("I1", 0, 0, signal=[EW_time, NS_time])

    for n1, n2 in [[W1, I1], [I1, E1]]:
        W.addLink(n1.name + n2.name, n1, n2, length=500, free_flow_speed=10, jam_density=0.2, signal_group=0)
        W.addLink(n2.name + n1.name, n2, n1, length=500, free_flow_speed=10, jam_density=0.2, signal_group=0)
    for n1, n2 in [[N1, I1], [I1, S1]]:
        W.addLink(n1.name + n2.name, n1, n2, length=500, free_flow_speed=10, jam_density=0.2, signal_group=1)
        W.addLink(n2.name + n1.name, n2, n1, length=500, free_flow_speed=10, jam_density=0.2, signal_group=1)

    return W


## Analyze World Function

Similar to the create_world function, the analyze_world function analyzes the traffic data after the simulation runs.

In [ ]:
@trace.bundle(trainable=False, allow_external_dependencies=True)
def analyze_world(W):
    assert not W.check_simulation_ongoing(), "Simulation has not completed."

    outputDict = {"Avg. Delay": W.analyzer.average_delay}
    time_lost = 0
    num_vehicles = 0

    for k, v in W.analyzer.od_trips.items():
        outputDict[k] = {"Trips attempted": v}
        num_vehicles += v
        outputDict[k]["Trips completed"] = W.analyzer.od_trips_comp[k]
        theoretical_minimum = W.analyzer.od_tt_free[k]
        observed_delay = np.sum(W.analyzer.od_tt[k] - theoretical_minimum)
        imputed_delay = (np.max(W.analyzer.od_tt[k]) + 1 - theoretical_minimum) * (v - len(W.analyzer.od_tt))
        time_lost += observed_delay + imputed_delay
        outputDict[k]["Time lost per vehicle"] = (observed_delay + imputed_delay) / v

    outputDict["Best-Case Estimated Delay"] = time_lost / num_vehicles
    variance = 0
    for k, v in W.analyzer.od_trips.items():
        variance += ((outputDict[k]["Time lost per vehicle"] - outputDict["Best-Case Estimated Delay"]) ** 2) * v

    score = outputDict["Best-Case Estimated Delay"] + np.sqrt(variance / num_vehicles)
    outputDict["OVERALL SCORE"] = score

    return outputDict

## Run Approach Function

This helper function runs the optimization approach, catching exceptions thrown if any as feedback to the `trace` optimizer.

In [ ]:
def run_approach(num_iter, trace_memory=0, trace_config="OAI_CONFIG_LIST", rng=None):
    W = None
    return_val = np.zeros((num_iter, 3))
    
    def traffic_simulation(EW_green_time, NS_green_time):
        W = None
        try:
            W = create_world(EW_green_time, NS_green_time)
        except Exception as e:
            e_node = ExceptionNode(
                e,
                inputs={"EW_green_time": EW_green_time, "NS_green_time": NS_green_time},
                description="[exception] Simulation raises an exception with these inputs.",
                name="exception_step",
            )
            return e_node
        W.data.exec_simulation()
        return_dict = analyze_world(W)
        return return_dict

    EW_x = trace.node(MIN_GREEN_TIME, trainable=True, constraint=f"[{MIN_GREEN_TIME},{MAX_GREEN_TIME}]")
    NS_x = trace.node(MIN_GREEN_TIME, trainable=True, constraint=f"[{MIN_GREEN_TIME},{MAX_GREEN_TIME}]")
    optimizer = FunctionOptimizerV2Memory(
                [EW_x, NS_x], memory_size=trace_memory, config_list=config_list_from_json(trace_config)
            )

    optimizer.objective = (
                "You should suggest values for the variables so that the OVERALL SCORE is as small as possible.\n"
                + "There is a trade-off in setting the green light durations.\n"
                + "If the green light duration for a given direction is set too low, then vehicles will queue up over time and experience delays, thereby lowering the score for the intersection.\n"
                + "If the green light duration for a given direction is set too high, vehicles in the other direction will queue up and experience delays, thereby lowering the score for the intersection.\n"
                + "The goal is to find a balance for each direction (East-West and North-South) that minimizes the overall score of the intersection.\n"
                + optimizer.default_objective
        )

    for i in range(num_iter):
        result = traffic_simulation(EW_x, NS_x)
        feedback = None
        if isinstance(result, ExceptionNode):
            return_val[i] = (EW_x.data, NS_x.data, np.inf)
            feedback = result.data
        else:
            return_val[i] = (EW_x.data, NS_x.data, result.data["OVERALL SCORE"])
            feedback = (
                "OVERALL SCORE: "
                + str(result.data["OVERALL SCORE"])
                + "\nPlease try to optimize the intersection further. If you are certain that you have found the optimal solution, please suggest it again."
            )

        optimizer.zero_feedback()
        optimizer.backward(result, feedback, visualize=True)
        optimizer.step(verbose=True)
    return return_val

## Running the Notebook

Now, you can run each cell of the notebook step-by-step to see how the simulation and optimization are performed. You can modify the parameters and observe the effects on the optimization process.

In [ ]:
demand_dict = create_demand(42, 0.25)
returned_val = run_approach(100, 5)
print(returned_val)

This completes the tutorial on using the Trace package for numerical optimization in a traffic simulation. Happy optimizing!